<a href="https://colab.research.google.com/github/ancestor9/24_fall_textmining_NLP/blob/main/1101_pycaret_gradio_%26_FastAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# 필요한 라이브러리 설치
!pip install pycaret --quiet

## **Without FastAPI**

In [21]:
from pycaret.classification import *
import pandas as pd
import joblib

# PyCaret을 이용해 데이터 준비 및 모델 학습
# 이 예제에서는 classification 모듈을 사용합니다.
data = pd.read_csv("https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/diabetes.csv")

# 열 이름 간소화 (호환성 문제 해결)
data.columns = [
    'Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
    'BMI', 'DiabetesPedigreeFunction', 'Age', 'Class'
]

# PyCaret 설정 초기화 및 모델 학습
env = setup(data, target='Class', session_id=123)  # 'Class'를 실제 데이터의 타겟 변수로 교체
tuned_model = create_model('rf')  # 랜덤 포레스트 모델 생성

# 모델 저장
save_model(tuned_model, 'diabetes_rf_model')

# 모델 로드
model = joblib.load("diabetes_rf_model.pkl")


,Description,Value
0,Session id,123
1,Target,Class
2,Target type,Binary
3,Original data shape,"(768, 9)"
4,Transformed data shape,"(768, 9)"
5,Transformed train set shape,"(537, 9)"
6,Transformed test set shape,"(231, 9)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7963,0.8820,0.5789,0.7857,0.6667,0.5248,0.5375
1,0.7593,0.8098,0.6316,0.6667,0.6486,0.4658,0.4661
2,0.7593,0.8782,0.4737,0.7500,0.5806,0.4236,0.4456
3,0.7037,0.7632,0.4737,0.6000,0.5294,0.3175,0.3223
4,0.8148,0.8496,0.6842,0.7647,0.7222,0.5840,0.5860
5,0.6852,0.6699,0.4211,0.5714,0.4848,0.2656,0.2720
6,0.7963,0.7850,0.6316,0.7500,0.6857,0.5367,0.5410
7,0.7925,0.8603,0.5000,0.8182,0.6207,0.4890,0.5171
8,0.6792,0.6833,0.4444,0.5333,0.4848,0.2548,0.2570


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Transformation Pipeline and Model Successfully Saved


## **예측**

In [22]:

# 예측 함수 정의
def predict(data: dict):
    # 입력 데이터로부터 DataFrame 생성
    input_data = pd.DataFrame([data])

    # 예측 수행
    prediction = model.predict(input_data)

    # 결과 반환
    return {"prediction": int(prediction[0])}

# 예측 함수 테스트
if __name__ == "__main__":
    # 테스트 데이터 예시
    test_data = {
        "Pregnancies": 6,
        "Glucose": 148,
        "BloodPressure": 72,
        "SkinThickness": 35,
        "Insulin": 0,
        "BMI": 33.6,
        "DiabetesPedigreeFunction": 0.627,
        "Age": 50
    }

    # 예측 수행 및 결과 출력
    print(f'새로운 입력값: {test_data}')
    print(pd.DataFrame([test_data]))
    result = predict(test_data)
    print(f'예측결과는 : {result}')


새로운 입력값: {'Pregnancies': 6, 'Glucose': 148, 'BloodPressure': 72, 'SkinThickness': 35, 'Insulin': 0, 'BMI': 33.6, 'DiabetesPedigreeFunction': 0.627, 'Age': 50}
   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   

   DiabetesPedigreeFunction  Age  
0                     0.627   50  
예측결과는 : {'prediction': 1}


# **[create_app](https://pycaret.gitbook.io/docs/get-started/functions/deploy)**
### **This function(creat_app) creates a basic gradio app for inference. It will later be expanded for other app types such [Streamlit](https://streamlit.io/).**

In [23]:
# load dataset
from pycaret.datasets import get_data
juice = get_data('juice')

# init setup
from pycaret.classification import *
exp_name = setup(data = juice,  target = 'Purchase')

# train a model
lr = create_model('lr')

# create app
create_app(lr)

,Id,Purchase,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,Store7,PctDiscMM,PctDiscCH,ListPriceDiff,STORE
0,1,CH,237,1,1.75,1.99,0.00,0.0,0,0,0.500000,1.99,1.75,0.24,No,0.000000,0.000000,0.24,1
1,2,CH,239,1,1.75,1.99,0.00,0.3,0,1,0.600000,1.69,1.75,-0.06,No,0.150754,0.000000,0.24,1
2,3,CH,245,1,1.86,2.09,0.17,0.0,0,0,0.680000,2.09,1.69,0.40,No,0.000000,0.091398,0.23,1
3,4,MM,227,1,1.69,1.69,0.00,0.0,0,0,0.400000,1.69,1.69,0.00,No,0.000000,0.000000,0.00,1
4,5,CH,228,7,1.69,1.69,0.00,0.0,0,0,0.956535,1.69,1.69,0.00,Yes,0.000000,0.000000,0.00,0


,Description,Value
0,Session id,1029
1,Target,Purchase
2,Target type,Binary
3,Target mapping,"CH: 0, MM: 1"
4,Original data shape,"(1070, 19)"
5,Transformed data shape,"(1070, 19)"
6,Transformed train set shape,"(749, 19)"
7,Transformed test set shape,"(321, 19)"
8,Numeric features,17
9,Categorical features,1


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8000,0.9244,0.8000,0.8000,0.7962,0.5714,0.5774
1,0.8667,0.9111,0.8667,0.8663,0.8658,0.7191,0.7202
2,0.8400,0.8741,0.8400,0.8390,0.8388,0.6583,0.6594
3,0.8933,0.9520,0.8933,0.8958,0.8939,0.7779,0.7791
4,0.7867,0.8681,0.7867,0.7845,0.7832,0.5385,0.5421
5,0.8000,0.8928,0.8000,0.7981,0.7977,0.5701,0.5722
6,0.7600,0.8133,0.7600,0.7600,0.7600,0.4940,0.4940
7,0.8267,0.9040,0.8267,0.8378,0.8286,0.6458,0.6519
8,0.8533,0.9310,0.8533,0.8529,0.8516,0.6848,0.6873


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://46819ede953ea57527.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## **[gradio](https://www.gradio.app/)로 직접 만들어보기**

In [24]:
# 필요한 라이브러리 설치
import gradio as gr

# 예측 함수 정의 (Gradio 용)
def predict_gradio(Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age):
    # 입력 데이터를 DataFrame으로 생성
    input_data = pd.DataFrame([{
        "Pregnancies": Pregnancies,
        "Glucose": Glucose,
        "BloodPressure": BloodPressure,
        "SkinThickness": SkinThickness,
        "Insulin": Insulin,
        "BMI": BMI,
        "DiabetesPedigreeFunction": DiabetesPedigreeFunction,
        "Age": Age
    }])

    # 예측 수행
    prediction = model.predict(input_data)

    # 결과 반환
    return int(prediction[0])

# Gradio 인터페이스 생성
gr_interface = gr.Interface(
    fn=predict_gradio,
    inputs=[
        gr.components.Number(label="Pregnancies"),
        gr.components.Number(label="Glucose"),
        gr.components.Number(label="BloodPressure"),
        gr.components.Number(label="SkinThickness"),
        gr.components.Number(label="Insulin"),
        gr.components.Number(label="BMI"),
        gr.components.Number(label="DiabetesPedigreeFunction"),
        gr.components.Number(label="Age")
    ],
    outputs=gr.components.Textbox(label="Prediction"),
    title="Diabetes Prediction",
    description="Enter the details to predict diabetes risk",
    flagging_options=["Prediction"]
)

# 예측 함수 테스트
def predict(data: dict):
    # 입력 데이터로부터 DataFrame 생성
    input_data = pd.DataFrame([data])

    # 예측 수행
    prediction = model.predict(input_data)

    # 결과 반환
    return {"prediction": int(prediction[0])}

# 예측 함수 테스트
if __name__ == "__main__":
    # 테스트 데이터 예시
    test_data = {
        "Pregnancies": 6,
        "Glucose": 148,
        "BloodPressure": 72,
        "SkinThickness": 35,
        "Insulin": 0,
        "BMI": 33.6,
        "DiabetesPedigreeFunction": 0.627,
        "Age": 50
    }

    # 예측 수행 및 결과 출력
    result = predict(test_data)
    print(result)


    # Gradio 인터페이스 실행
gr_interface.launch()


{'prediction': 1}
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://574b1dd93147256c43.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## **With FastAPI**

In [15]:
# 필요한 라이브러리 설치
# !pip install pycaret fastapi uvicorn

from pycaret.classification import *
from fastapi import FastAPI
import pandas as pd
import joblib

# PyCaret을 이용해 데이터 준비 및 모델 학습
# 이 예제에서는 classification 모듈을 사용합니다.
data = pd.read_csv("https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/diabetes.csv")

# 열 이름 간소화 (호환성 문제 해결)
data.columns = [
    'Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
    'BMI', 'DiabetesPedigreeFunction', 'Age', 'Class'
]

# PyCaret 설정 초기화 및 모델 학습
env = setup(data, target='Class', silent=True, session_id=123)  # 'Class'를 실제 데이터의 타겟 변수로 교체
tuned_model = create_model('rf')  # 랜덤 포레스트 모델 생성

# 모델 저장
save_model(tuned_model, 'diabetes_rf_model')

# FastAPI 인스턴스 생성
app = FastAPI()

# 모델 로드
model = joblib.load("diabetes_rf_model.pkl")

# 예측 함수 정의
@app.post("/predict")
async def predict_api(data: dict):
    # 입력 데이터로부터 DataFrame 생성
    input_data = pd.DataFrame([data])

    # 예측 수행
    prediction = model.predict(input_data)

    # 결과 반환
    return {"prediction": int(prediction[0])}

# 예측 함수 테스트
def predict(data: dict):
    # 입력 데이터로부터 DataFrame 생성
    input_data = pd.DataFrame([data])

    # 예측 수행
    prediction = model.predict(input_data)

    # 결과 반환
    return {"prediction": int(prediction[0])}

# 예측 함수 테스트
if __name__ == "__main__":
    # 테스트 데이터 예시
    test_data = {
        "Pregnancies": 6,
        "Glucose": 148,
        "BloodPressure": 72,
        "SkinThickness": 35,
        "Insulin": 0,
        "BMI": 33.6,
        "DiabetesPedigreeFunction": 0.627,
        "Age": 50
    }

    # 예측 수행 및 결과 출력
    result = predict(test_data)
    print(result)

    # FastAPI 서버 실행 (uvicorn을 통해 실행)
    # uvicorn main:app --reload


TypeError: setup() got an unexpected keyword argument 'silent'